### Library Import

In [1]:
import os
from typing import Any, List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import lightgbm as lgb
import optuna
from sklearn.ensemble import VotingRegressor
from data_preprocessing import *
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


### Data Load

In [2]:
# 파일 호출
data_path: str = "/data/ephemeral/home/BTC/data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [3]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|██████████| 107/107 [00:03<00:00, 32.45it/s]


### Feature engineering

In [4]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close": "close",
    "hourly_network-data_difficulty_difficulty": "difficulty",
    "hourly_network-data_supply_supply_total": "supply_total",
    "hourly_network-data_utxo-count_utxo_count": "utxo_count",
    "hourly_network-data_velocity_velocity_supply_total": "velocity_supply_total"
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

(11552, 10)

In [5]:
# continuous 열을 따로 할당해둠
conti_cols: List[str] = [
    "close",
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count",
    "funding_rates",
    "velocity_supply_total"
]

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 6)]
)

# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [6]:
df

,ID,target,_type,funding_rates,open_interest,close,difficulty,supply_total,utxo_count,velocity_supply_total,...,funding_rates_1,funding_rates_2,funding_rates_3,funding_rates_4,funding_rates_5,velocity_supply_total_1,velocity_supply_total_2,velocity_supply_total_3,velocity_supply_total_4,velocity_supply_total_5
0,2023-01-01 00:00:00,2.0,train,0.005049,6.271344e+09,16536.747967,3.536407e+13,1.924871e+07,83308092.0,65.978971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 01:00:00,1.0,train,0.005049,6.288683e+09,16557.136536,3.536407e+13,1.924874e+07,83314883.0,65.977755,...,0.005049,NaN,NaN,NaN,NaN,65.978971,NaN,NaN,NaN,NaN
2,2023-01-01 02:00:00,1.0,train,0.005049,6.286796e+09,16548.149805,3.536407e+13,1.924879e+07,83314090.0,65.975570,...,0.005049,0.005049,NaN,NaN,NaN,65.977755,65.978971,NaN,NaN,NaN
3,2023-01-01 03:00:00,1.0,train,0.005067,6.284575e+09,16533.632875,3.536407e+13,1.924882e+07,83326258.0,65.975092,...,0.005049,0.005049,0.005049,NaN,NaN,65.975570,65.977755,65.978971,NaN,NaN
4,2023-01-01 04:00:00,2.0,train,0.006210,6.291582e+09,16524.712159,3.536407e+13,1.924886e+07,83339168.0,65.969825,...,0.005067,0.005049,0.005049,0.005049,NaN,65.975092,65.975570,65.977755,65.978971,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,test,NaN,1.486836e+10,NaN,8.810419e+13,NaN,179820708.0,15.708903,...,NaN,NaN,NaN,NaN,NaN,15.707708,15.707913,15.708644,15.709642,15.709948
11548,2024-04-26 04:00:00,NaN,test,NaN,NaN,NaN,8.810419e+13,NaN,179833897.0,15.709585,...,NaN,NaN,NaN,NaN,NaN,15.708903,15.707708,15.707913,15.708644,15.709642
11549,2024-04-26 05:00:00,NaN,test,NaN,NaN,NaN,8.810419e+13,NaN,179851249.0,15.709207,...,NaN,NaN,NaN,NaN,NaN,15.709585,15.708903,15.707708,15.707913,15.708644
11550,2024-04-26 06:00:00,NaN,test,NaN,NaN,NaN,8.810419e+13,NaN,179852452.0,15.710891,...,NaN,NaN,NaN,NaN,NaN,15.709207,15.709585,15.708903,15.707708,15.707913


In [7]:
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Train

In [8]:
X_train = train_df.drop(["ID", "target", "close"], axis=1)
y_train = train_df["close"]
target = train_df["target"]

In [9]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close['close'] = series
    close['close_lag1'] = close['close'].shift(1)
    close['close_lag1_percent'] = (close['close'] - close['close_lag1']) / close['close_lag1']
    close['class'] = close['close']
    for i in range(close.shape[0]):
        if close.loc[i, 'close_lag1_percent'] < -0.005:
            close.loc[i, 'class'] = 0
        elif close.loc[i, 'close_lag1_percent'] < 0:
            close.loc[i, 'class'] = 1
        elif close.loc[i, 'close_lag1_percent'] < 0.005:
            close.loc[i, 'class'] = 2
        else:
            close.loc[i, 'class'] = 3
            
    return close["class"].shift(-1).fillna(method="ffill")

In [10]:
# 모델 평가
def evaluate(valid_target: pd.Series, 
             y_valid: pd.Series, 
             y_pred: np.ndarray, 
             metric: str
) -> float:
    """평가지표 metric을 반환하는 함수입니다.

    Args:
        valid_target: (pd.Series): k-fold로 분할한 target의 검증 데이터
        y_valid (pd.Series): k-fold로 분할한 close의 검증 데이터
        y_pred (np.ndarray): 모델을 사용하여 예측한 변수
        metric (str): 사용할 평가지표 metric 이름

    Returns:
        float: 사용할 평가지표 metric 값
    """
    if metric == "accuracy":
        y_pred_class = close_to_class(y_pred)
        return accuracy_score(valid_target, y_pred_class)
    elif metric == "mae":
        return mean_absolute_error(y_valid, y_pred)
    elif metric == "mse":
        return mean_squared_error(y_valid, y_pred)
    elif metric == "mape":
        return mean_absolute_percentage_error(y_valid, y_pred)

In [11]:
def model_train(model: Any, 
                X_train: pd.DataFrame, 
                y_train: pd.Series, 
                cv: int, 
                metric: str, 
) -> Tuple[Any, float]:
    """K-Fold로 데이터를 분할한 후 전처리를 거쳐 주어진 모델로 데이터를 학습 및 평가를 진행합니다.

    Args:
        model (Any): 사용하는 모델 객체
        X_train (pd.DataFrame): 설명변수로 이루어진 학습 데이터프레임
        y_train (pd.Seris): 예측변수로 이루어진 학습 시리즈
        cv (int): 교차검증시 분할할 폴드의 수
        metric (str): 사용할 평가지표 metric 이름

    Returns:
        Any, float: 폴드 내에서 가장 평가지표 값이 높은 모델 객체, 평가지표 metric 값
    """
    kfold = KFold(n_splits=cv)
    score_list = []
    fold_model = []
    
    # warm_start는 모델의 속성으로, 같은 모델을 반복 학습할 때 이전 학습에서 학습된 파라미터를 초기화하지 않고 이어서 학습을 진행하는 옵션
    if hasattr(model, "warm_start"):
        model.warm_start = True

    # K-Fold 교차 검증
    for train_index, valid_index in kfold.split(X_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_valid, y_valid = X_train.iloc[valid_index], y_train.iloc[valid_index]

        valid_target = target[valid_index]
        
        # 전처리
        fill_feature(X_train_fold, method="mean")
        fill_feature(X_valid, method="mean")        
        
        # 모델 학습
        model.fit(X_train_fold, y_train_fold)
        fold_model.append(model)

        y_pred = model.predict(X_valid)
        score = evaluate(valid_target, y_valid, y_pred, metric=metric)  # 평가지표 metric 반환
        score_list.append(score)
    
    return fold_model[np.argmax(score_list)], np.max(score_list)

### Model Ensemble

In [12]:
# Lasso에만 사용할 피처
cols_dict_for_lasso = [
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count",
    "funding_rates",
    "velocity_supply_total"
]

In [13]:
# shift_feature로 생성된 컬럼들을 cols_dict_for_lasso에 추가
for col in conti_cols:
    for interval in range(1, 6):
        cols_dict_for_lasso.append(f"{col}_{interval}")

In [14]:
cols_dict_for_lasso

['open_interest',
 'difficulty',
 'supply_total',
 'utxo_count',
 'funding_rates',
 'velocity_supply_total',
 'close_1',
 'close_2',
 'close_3',
 'close_4',
 'close_5',
 'open_interest_1',
 'open_interest_2',
 'open_interest_3',
 'open_interest_4',
 'open_interest_5',
 'difficulty_1',
 'difficulty_2',
 'difficulty_3',
 'difficulty_4',
 'difficulty_5',
 'supply_total_1',
 'supply_total_2',
 'supply_total_3',
 'supply_total_4',
 'supply_total_5',
 'utxo_count_1',
 'utxo_count_2',
 'utxo_count_3',
 'utxo_count_4',
 'utxo_count_5',
 'funding_rates_1',
 'funding_rates_2',
 'funding_rates_3',
 'funding_rates_4',
 'funding_rates_5',
 'velocity_supply_total_1',
 'velocity_supply_total_2',
 'velocity_supply_total_3',
 'velocity_supply_total_4',
 'velocity_supply_total_5']

In [15]:
cols_dict_for_lgb = [
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count",
    "funding_rates",
    "velocity_supply_total"
]

In [16]:
# shift_feature로 생성된 컬럼들을 cols_dict_for_lgb에 추가
for col in conti_cols:
    for interval in range(1, 6):
        cols_dict_for_lgb.append(f"{col}_{interval}")

In [17]:
cols_dict_for_xgb = [
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count",
    "funding_rates",
    "velocity_supply_total"
]

In [18]:
# shift_feature로 생성된 컬럼들을 cols_dict_for_xgb에 추가
for col in conti_cols:
    for interval in range(1, 6):
        cols_dict_for_xgb.append(f"{col}_{interval}")

In [19]:
xgb_params = {
    'n_estimators': 159,
    'learning_rate': 0.044545369253400344,
    'max_depth': 7,
    'min_child_weight': 5,
    'colsample_bytree': 0.5878501242431816,
    'subsample': 0.7729401646786744,
    "booster": "gbtree",
    "device": "gpu",
    "random_state": 42
}
lgb_params = {
    "verbose" : -1,
    'num_leaves': 77,
    'learning_rate': 0.019576305246645095,
    'n_estimators': 294,
    'max_depth': 8,
    'min_child_weight': 1,
    'subsample': 0.6520270159934872,
    'colsample_bytree': 0.5000782016959966,
    "random_state" : 42
}

In [20]:
# 파이프라인 구성
lasso_pipeline = Pipeline([
    ('feature_select', ColumnTransformer([("select", "passthrough", cols_dict_for_lasso)], remainder="drop")),
    #('scaler', StandardScaler()),
    ('lasso', Lasso(alpha=20, max_iter=2000, random_state=42))
])

xgb_pipeline = Pipeline([
    ('feature_select', ColumnTransformer([("select", "passthrough", cols_dict_for_xgb)], remainder="drop")),
    ('xgb', XGBRegressor(**xgb_params))
])

lgb_pipeline = Pipeline([
    ('feature_select', ColumnTransformer([("select", "passthrough", cols_dict_for_lgb)], remainder="drop")),
    ('lgb', lgb.LGBMRegressor(**lgb_params))
])

In [21]:
def objective(trial):
    weight_xgb = trial.suggest_float("weight_xgb", 0.1, 3.0)
    weight_lgb = trial.suggest_float("weight_lgb", 0.1, 3.0)
    weight_lr = trial.suggest_float("weight_lr", 0.1, 3.0)
    
    ensemble_model = VotingRegressor(
        estimators=[
            ('xgb', xgb_pipeline),
            ('lgb', lgb_pipeline),
            ('lr', lasso_pipeline)
        ],
        weights=[weight_xgb, weight_lgb, weight_lr]
    )
    
    _, score = model_train(ensemble_model, X_train, y_train, cv=5, metric="accuracy")
    return score

In [22]:
# Optuna로 가중치 최적화
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-09-26 07:41:39,273] A new study created in memory with name: no-name-88df6368-0ef0-4f8d-bc78-06c6a13c1d28
[I 2024-09-26 07:41:50,029] Trial 0 finished with value: 0.4737442922374429 and parameters: {'weight_xgb': 1.3246474231683865, 'weight_lgb': 0.2464716649562725, 'weight_lr': 1.0716448384956607}. Best is trial 0 with value: 0.4737442922374429.
[I 2024-09-26 07:42:01,026] Trial 1 finished with value: 0.4617579908675799 and parameters: {'weight_xgb': 1.038072864608512, 'weight_lgb': 0.7909395729385074, 'weight_lr': 0.5567823308609487}. Best is trial 0 with value: 0.4737442922374429.
[I 2024-09-26 07:42:11,680] Trial 2 finished with value: 0.4714611872146119 and parameters: {'weight_xgb': 2.401021959287184, 'weight_lgb': 2.2965395507957767, 'weight_lr': 1.906751797519744}. Best is trial 0 with value: 0.4737442922374429.
[I 2024-09-26 07:42:22,323] Trial 3 finished with value: 0.4863013698630137 and parameters: {'weight_xgb': 1.0065285958157553, 'weight_lgb': 0.3052064212017753,

In [23]:
# 최적 가중치를 사용한 최종 모델 구성
best_weights = [
    study.best_params["weight_xgb"],
    study.best_params["weight_lgb"],
    study.best_params["weight_lr"]
]

ensemble_model = VotingRegressor(
    estimators=[('xgb', xgb_pipeline), ('lgb', lgb_pipeline), ('lr', lasso_pipeline)],
    weights=best_weights
)


In [24]:
# 학습
Best_ensemble_model, model_accuracy = model_train(ensemble_model, X_train, y_train, cv=5, metric="accuracy")
print(f"Ensemble model accuracy: {model_accuracy}")

Ensemble model accuracy: 0.4948630136986301


### Inference

In [25]:
# fold_best_xgb_model이 학습 데이터 전체를 학습할 수 있도록 결측치 처리
X_train = fill_feature(X_train, method="mean")
X_test = test_df.drop(["ID", "target", "close"], axis=1)
X_test = fill_feature(X_test, method="mean")

In [26]:
Best_ensemble_model.fit(X_train, y_train)
y_test_pred = Best_ensemble_model.predict(X_test)
y_test_pred_class = close_to_class(y_test_pred)

In [27]:
# lr output
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("output.csv", index=False)

In [28]:
submission_df['target'].value_counts()

target
2    1367
1    1296
3      72
0      57
Name: count, dtype: int64